# Creating a datase out of only perfect rated answers
This code can be used to achieve this, but this is not part of the thesis

In [1]:
import datetime
import json
import collections
import pymongo

In [2]:
file_path = '../keys.json'
target_key = 'mongo_path'
with open(file_path, 'r') as file:
    data = json.load(file)
cluster_path = data.get(target_key)
if cluster_path is not None:
    print(f"Cluster Path found")


Cluster Path found


In [3]:

# Establish a connection to the MongoDB cluster
cluster = pymongo.MongoClient(cluster_path)

# Connect to a specific database
db = cluster["DialoKEY"]

# Read a document from a collection
collection = db["logs_04"]
collection_cleaned = db["logs_cleaned_3"]
document = collection.find_one()

In [4]:
feeds2 = []
feeds = collections.defaultdict(list)
dt = datetime.datetime.now()
timestamp = dt.strftime('%Y_%m_%d_%H:%M:%S')[:-3]
cursor4 = collection_cleaned.find()
filename = f"./output_files/{timestamp}_perfect_training.json"
history_counter = ({
    "0": 0,
    "1": 0,
    "2": 0,
    '3': 0,
        '4': 0
})
for document in cursor4:

    if 'Perfect' not in document['User Flag']:
        continue
    with open(filename, mode='w', encoding='utf-8') as feedsjson:
        history = ""
        for entries in document['Question'][:-1]:
            history += entries + "\n"
        history_counter[str(int(len(document['Question']) / 2))] += 1
        entry2 = {'prompt': f"{history}\nKeywords: {document['Keyword (human)']}\nAnswer:\n\n###\n\n",
                  'completion': f"{document['Answers']['Answer Created']} END"}
        feeds['all'].append(entry2)
        json.dump(feeds['all'], feedsjson)

print(history_counter)

{'0': 0, '1': 240, '2': 242, '3': 246, '4': 2}
